FIAP - Tecnólogo em Inteligência Artificial

<img src="https://www.spdm.org.br/media/k2/items/cache/6da4961fc61d08e9b5f80c8ea30d8ffc_XL.jpg" alt="Girl in a jacket" width="300" align="right">

**Entrega**: XX/XX/2022, via Teams - IMPORTANTE: renomeie o notebook incluindo seu nome! 


# Checkpoint #5 - Classificação de Imagens com CNNs

## O problema 

O carcinoma ductal invasivo (IDC), também conhecido como carcinoma ductal infiltrante, é um câncer que começou a crescer em um ducto de leite e invadiu o tecido fibroso ou gorduroso da mama fora do ducto. IDC é **a forma mais comum de câncer de mama**, representando 80% de todos os diagnósticos de câncer de mama.

Sua *taxa de sobrevivência* de 5 anos é bastante alta --- quase 100% quando o tumor é detectado e tratado precocemente [fonte](https://www.cancer.org/cancer/breast-cancer/understanding-a-breast-cancer-diagnosis/breast-cancer-survival-rates.html). Ou seja, uma pesssoa com câncer identificado e tratado possui quase 100% de chances de viver nos próximos 5 anos de uma pessoa que não teve o câncer. 

O *prognóstico* depende do momento que o câncer foi identificado e seu *estadiamento* (processo para determinar a localização e a extensão do câncer no corpo de uma pessoa, ou seja, é a forma de determinar o avanço da doença no organismo de um paciente).

A forma de tratar o câncer depende do estadiamento e do seu tipo. 


## O desafio

Neste checkpoint seu grupo (utilizem a formação que desejarem) será desafiado a construir um classificador automático de câncer de mama por meio da análise de imagens. 

Vocês **devem adotar 2 abordagens distintas e compará-las** em termos de métricas de performance do modelo¹: 

1. treinar um modelo do zero (*from scratch*), com a arquitetura que desejarem usando CNNs;
2. treinar um modelo com *transfer learning* e/ou *fine tuning* a partir de uma rede pré-treinada no Imagenet².

A entrega do trabalho deve ser feita com o notebook executado e com células markdown narrando as decisões de design do grupo, bem como as justificativas. Não deixem de encerrarem com uma (breve!) conclusão sobre o trabalho.

OBSERVAÇÕES:

¹ vocês podem usar as métricas que quiserem desde que sejam métricas pertinentes para problemas de classificação. A comparação entre uma abordagem e outra deve fazer sentido! (Lembrem-se das boas práticas de Machine Learning)

² o TensorFlow possui diversas arquiteturas já treinadas na Imagenet. Vocês podem escolher a que desejarem, *justificando a escolha* (performance? tempo? etc.)


## Os dados

Vamos utilizar um dataset de imagens histopatológicas de mama. [Imagens histopatológicas](https://www.mediallabor.com.br/exame-histopatologico) são aquelas geradas com auxílio de um microscópico para analisar a composição das células de um tecido.

><font color="red">NOTA 1: </font> *claramente nosso classificador automático não é recomendado para **triagem** (detecção precoce) do câncer de mama, afinal, para coletarmos a imagem precisamos fazer uma biópsia da lesão, ou seja, é um procedimento invasivo! A biópsia só é feita em caso de alta suspeita de malignidade de uma lesão*. 

><font color="red">NOTA 2: </font> *então qual é o interesse deste tipo de dado? Como dissemos acima, este é o tipo de câncer de mama mais comum, porém existem outros! Nosso classificador não será útil na detecção, mas como identificação do tipo de câncer, para guiar estadiamento e tratamento*. 

<font color="CC0099">E acima de tudo, nosso maior objetivo é praticar uso de Deep Learning em imagens reais! </font> 


### Dataset

Nosso dataset virá de uma [competição do Kaggle](https://www.kaggle.com/paultimothymooney/breast-histopathology-images). Vocês podem encontrar mais informações sobre a descrição dos dados lá. 

### Como baixar o dataset? 

Recomendo que vocês utilizem o Colab para este trabalho, devido a presença de GPUs. Para acessar os dados vocês possuem 3 alternativas: 

1. Baixar os dados localmente, subindo uma cópia no seu Google Drive, fazer uma ligação (mount) do Colab com o Google Drive para ler as imagens de lá. Aqui o procedimento de download/upload só será feito uma vez, mas você precisa adaptar o código abaixo para fazer isso;  

2. Usar o código abaixo para baixar o dataset diretamente do Kaggle! Para isso vocês precisam: 

  a) ter uma conta no Kaggle [veja](https://www.kaggle.com/)

  b) baixar o arquivo .json com suas credenciais no seu computador (veja o tutorial da parte "API credentials" [deste link](https://github.com/Kaggle/kaggle-api))

  c) rodar os códigos abaixo para baixar o dataset no seu ambiente de desenvolvimento na nuvem. Tudo começa rodando a primeira célula na qual você abre um diálogo, indica onde está no seu computador o arquivo *kaggle.json* e subindo-o para o Colab. A código "instala" o arquivo automaticamente. 

  Nota: lembre-se que o Colab mata seus dados toda vez que encerra a execução, então para quem usar esta abordagem, precisa baixar e preparar o dataset toda vez que for começar a programar. 

3. Fazer uma mescla das 2 abordagens anteriores, baixando o arquivo via kaggle e salvando no seu Google Drive. Vocês já estão grandinhos para isso! =)




## Mão na massa! 

Aqui estou seguindo a abordagem **2.** para obter os dados. 
 



In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


Aqui temos apenas um comando sendo executado para verificar se tudo deu certo. 

In [2]:
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/world-population-dataset                     World Population Dataset                             17KB  2022-08-31 11:20:04           6282        201  1.0              
kaggleashwin/population-dataset                                World Population by Countries Dataset (1960-2021)    69KB  2022-08-31 05:30:26           1661         71  1.0              
iamsouravbanerjee/house-rent-prediction-dataset                House Rent Prediction Dataset                        82KB  2022-08-20 13:49:03          12247        309  1.0              
pantanjali/unemployment-dataset                                Un

Agora sim, **baixando o dataset** no nosso ambiente de desenvolvimento. 

In [3]:
!kaggle datasets download "paultimothymooney/breast-histopathology-images"

 99% 3.08G/3.10G [01:12<00:00, 66.9MB/s]
100% 3.10G/3.10G [01:12<00:00, 45.8MB/s]


Crie uma pasta *data* e descompacte o dataset nela.

In [4]:
!mkdir data
!unzip breast-histopathology-images.zip -d data 

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: data/IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: data/IDC_regular_ps50_idx5

## Alguns imports, alterem conforme necessidade

In [5]:
from glob import glob
import random
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras

from keras.utils.np_utils import to_categorical

## Preparando os dados para uso

Descubra os nomes das imagens da classe 0 (negativa, não câncer) e da classe 1 (positiva, câncer). 

In [6]:
imagePatches = glob("/content/data/IDC_regular_ps50_idx5/**/*.png", 
                    recursive=True)

class0 = []
class1 = []

for filename in imagePatches:
  if filename.endswith("class0.png"):
    class0.append(filename)
  else:
    class1.append(filename)

Diminua o tamanho do dataset, pois não queremos demorar demais no treinamento. 

In [7]:
nClass0 = len(class0)
nClass1 = len(class1)

print(f"Existem {nClass0} exemplos negativos e {nClass1} exemplos positivos.")
print(f"Vamos usar apenas uma amostragem igualitária de cada classe!")
print(f"Então nosso dataset final deve ter {nClass1} exemplos de cada classe.")

sampled_class0 = random.sample(class0, nClass1)
sampled_class1 = random.sample(class1, nClass1)

print(f"Dataset final: ")
print(f"  - N classe negativa: {len(sampled_class0)}")
print(f"  - N classe positiva: {len(sampled_class1)}")

Existem 198738 exemplos negativos e 78786 exemplos positivos.
Vamos usar apenas uma amostragem igualitária de cada classe!
Então nosso dataset final deve ter 78786 exemplos de cada classe.
Dataset final: 
  - N classe negativa: 78786
  - N classe positiva: 78786


Cria um array de tuplas para cada classe, sendo que a tupla está no formato `(imagem, label)`, sendo:
  - `imagem`: a imagem no formato de matriz de pixels
  - `label`: int indicando o label (0 ou 1)

In [8]:
from matplotlib.image import imread
import cv2

def get_image_arrays(data, label):
  img_arrays = []
  for i in data:
    if i.endswith(".png"):
      img = cv2.imread(i, cv2.IMREAD_COLOR)
      img_sized = cv2.resize(img, (50, 50), interpolation=cv2.INTER_LINEAR)
      img_arrays.append([img_sized, label])
  return img_arrays 

class0_array = get_image_arrays(sampled_class0, 0)
class1_array = get_image_arrays(sampled_class1, 1)

Junte os dados numa coisa só, e misture, mesclando as imagens para trazer aleatoriedade na seleção.

In [9]:
all_data = np.concatenate((class0_array, class1_array))
random.seed(42)
random.shuffle(all_data)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Separe os dados (pixels), dos labels (ints). Devemos terminar com 157572 imagens, escaladas em 50x50 pixels, sendo que cada imagem é expressa em 3 canais (RBG).

In [10]:
X = []
y = []
for image, label in all_data:
  X.append(image)
  y.append(label)

X = np.array(X).reshape(-1, 50, 50, 3)
X.shape

(157572, 50, 50, 3)

Já estou dando para vocês a divisão de TREINO e TESTE, sendo 75%/25%, respectivamente. O shape final deve ser algo +/- assim:

  > `(118179, 50, 50, 3) (39393, 50, 50, 3) (118179, 2) (39393, 2)`

Apesar de ser um problema de classificação binária, vamos colher as predições e tratar os labels como pequenos arrays de 2 posições, para obtermos as probabilidades de cada exemplo pertencer a cada uma das classes. Por isso o `to_categorical()`. Não se preocupem, os códigos do TensorFlow/Keras são robustos para trabalhar desta forma (inclusive já fizemos isso em sala).



In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(118179, 50, 50, 3) (39393, 50, 50, 3) (118179, 2) (39393, 2)


## e agora??? 

Agora é com vocês! Bom trabalho! =) 

DICAS: 
- Minha avaliação não será baseada nas melhores métricas obtidas, mas na completude do projeto. Se quiserem limitar o tamanho do dataset para acelerar a codificação/treinamento, tudo bem!
- Deixei o código pronto para escalar as imagens para 50x50 pixels. Se quiserem outra resolução, vão em frente!
- Na hora de fazerem transfer learning e/ou fine tuning, lembrem-se que cada arquitetura possui uma resolução preferida de entrada. Consultem na internet/documentação àquela da arquitetura escohida. 
- Vocês podem gostar [deste tutorial](https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/) para usar transfer com datasets com dimensões diferentes daquela original da rede.
- Vejam o exemplo da [documentação da VGG16](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16) para conhecerem os parâmetros a serem alterados na rede.
- A normalização das imagens para alimentar a rede passa por excluir o "valor médio" do pixel do dataset. Para transfer isso é feito com funções auxiliares de cada arquitetura. Revejam os tutoriais que usamos em sala para recaptularem como fazer. Para treinamento do zero (from scratch), vocês podem apenas normalizar os pixels num intervalo [0, 1]. Também fizemos isso em alguns tutoriais. 

## CNN

In [12]:
# Import 
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [13]:
# Definir a arquitetura da CNN -> camada de extração de features
hand_model = Sequential()
hand_model.add( Rescaling(1./255, input_shape=(50, 50, 3)))

hand_model.add( layers.Conv2D(16, kernel_size=10, activation="relu") )
hand_model.add( layers.MaxPooling2D(3) ) 

hand_model.add( layers.Conv2D(32, kernel_size=8, activation="relu") )
hand_model.add( layers.MaxPooling2D(2) ) 


In [14]:
# Definir a arquitetura da CNN -> camada de decisão (classificação)
hand_model.add( layers.Flatten() )
hand_model.add( layers.Dense(50, activation="relu") )
hand_model.add( layers.Dense(20, activation="relu") )
hand_model.add( layers.Dense(2, activation="sigmoid") )

In [15]:
# Compila o modelo
hand_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [16]:
# Definir a estratégia de Early Stopping
es = EarlyStopping(monitor="val_accuracy", 
                   patience=10, 
                   restore_best_weights=True)

In [17]:
# Treinamento do modelo 
with tf.device("/device:GPU:0"):
  hand_model.fit(X_train, y_train,
                 epochs=50,
                 validation_split=0.2,
                 batch_size=32,
                 callbacks=[es])

Epoch 1/50
2955/2955 [==============================] - 27s 5ms/step - loss: 0.3412 - accuracy: 0.8607 - val_loss: 0.3218 - val_accuracy: 0.8669
Epoch 2/50
2955/2955 [==============================] - 14s 5ms/step - loss: 0.3190 - accuracy: 0.8694 - val_loss: 0.3125 - val_accuracy: 0.8668
Epoch 3/50
2955/2955 [==============================] - 15s 5ms/step - loss: 0.3141 - accuracy: 0.8722 - val_loss: 0.3370 - val_accuracy: 0.8665
Epoch 4/50
2955/2955 [==============================] - 15s 5ms/step - loss: 0.3121 - accuracy: 0.8736 - val_loss: 0.3092 - val_accuracy: 0.8716
Epoch 5/50
2955/2955 [==============================] - 15s 5ms/step - loss: 0.3103 - accuracy: 0.8750 - val_loss: 0.3048 - val_accuracy: 0.8750
Epoch 6/50
2955/2955 [==============================] - 15s 5ms/step - loss: 0.3079 - accuracy: 0.8763 - val_loss: 0.3244 - val_accuracy: 0.8731
Epoch 7/50
2955/2955 [==============================] - 15s 5ms/step - loss: 0.3049 - accuracy: 0.8782 - val_loss: 0.3132 - val_ac

In [18]:
# Avaliação do modelo
loss, acc = hand_model.evaluate(X_test, y_test)

print(f"A loss do modelo é {loss:.2f} e a ACC é: {acc:.2f}")

1232/1232 [==============================] - 4s 3ms/step - loss: 0.3351 - accuracy: 0.8862
A loss do modelo é 0.34 e a ACC é: 0.89
